# Bac à sable

## Check header CSV source

In [33]:
import os.path
import pandas as pd

### Paramètres 

In [73]:
UPLOAD_FOLDER = 'uploads'
DOWNLOAD_FOLDER = 'downloads'
EXPORT_FILENAME = 'points.scr'
ALLOWED_EXTENSIONS = {'csv'}
SOURCE_FILE = '../static/data_sample/profile.csv'
CSV_COLUMS_PROFILE = ['distance', 'bathymetry', 'mns', 'mnt', 'x', 'y']
CSV_COLUMS_LIDAR = ['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']

### Fonctions de contrôle

In [69]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def csv_header_check(df):
    source_colums = get_csv_header(df)
    if source_colums == CSV_COLUMS_PROFILE or source_colums == CSV_COLUMS_LIDAR:
        return True
    else:
        return False

def get_csv_header(df):
    return df.columns

def get_csv_type_profile(df):
    if csv_header_check(df) :
        source_colums = get_csv_header(df)
        if source_colums == CSV_COLUMS_PROFILE:
            return 'profile'
        else:
            return 'lidar'
    else:
        return


### Tests des fonctions

#### Nom du fichier

In [64]:
print(SOURCE_FILE.split('/')[-1])

LIDAR_profile.csv


#### Contrôle de l'extension CSV

In [74]:
file = SOURCE_FILE
file_name = file.split('/')[-1]
if allowed_file(file_name):
    print("File type pass")

File type pass


#### Chargement du fichier CSV dans le dataframe

In [66]:
df = pd.read_csv(file)

#### Récupère les noms de colonnes dans une liste

In [67]:
print(df.columns.to_list())

['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']


#### Affiche les valeurs attendues pour profil et lidar

In [75]:
print(CSV_COLUMS_PROFILE)
print(CSV_COLUMS_LIDAR)

['distance', 'bathymetry', 'mns', 'mnt', 'x', 'y']
['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']


#### Affiche si les colonnes du fichier sont celles attendues pour profile et lidar

In [77]:
print(csv_header_check_profile(df))
print(csv_header_check_lidar(df))

False
True


#### Affiche le type de profil du fichier 

In [72]:
print(get_csv_type_profile(df))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()